# Code for calculating the final dataframes for regression of the impact of the flow of momentum on managerial decision-making.

In [ ]:
#ten-minute time windows for pitch control and PCV
import os
from progressbar import ProgressBar
import pandas as pd
import pickle

#create list with all files used for pitch control and PCV
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []
for file in os.listdir(path):
    if 'pc_over_last_five_minutes' in file:
        pc_matches.append(file[25:32])

#create empty dataframe
final_df = pd.DataFrame()
pbar = ProgressBar()

#iterate over each match
for game in pbar(pc_matches):
    #open pitch control and PCV file
    infile = open('pc_over_last_five_minutes' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    
    #create first and second half dataframe
    fh = df_pc.loc[df_pc['Half'] == 1]
    sh = df_pc.loc[df_pc['Half'] == 2]
    
    #open match decision dataframe
    infile = open('Decision matches dataframe.p','rb')
    df_decisions = pickle.load(infile)
    infile.close()
    
    #take only second half decisions for each match
    df_decisions = df_decisions.loc[df_decisions['Match'] == game]
    df_decisions = df_decisions.loc[df_decisions['half'] == '2']
    df_decisions = df_decisions[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'formation_change', 'home score', 'away score', 'current_formation_home', 'current_formation_away', 'Riskyness decision', 'Match']]
    df_decisions['minute'] = pd.to_numeric(df_decisions['minute'])
    df_decisions = df_decisions.loc[df_decisions['minute'] >= 50]

    
    mean_pc_b_home = []
    mean_wpc_b_home = []
    mean_pc_b_away = []
    mean_wpc_b_away = []
    index_min_below = []
    index_min_above = []
    indexa = []
    indexb = []
    
    #for each row in the decision dataframe: add the pitch control and pcv values 10 minutes before the decision
    
    for i in range(len(df_decisions)):
        minute = float(df_decisions.iloc[i]['minute'])
        second = float(df_decisions.iloc[i]['seconds'])
        index_above = sh[sh['Time'] > float(minute + second/60)].iloc[0].name
        try:
            index_below = sh[sh['Time'] <= float(minute + second/60)].iloc[-1].name
        except:
            index_below = index_above
        index_5min_above = sh.index[sh.index <= index_above +5*60*25].max()
        index_5min_below = sh.index[sh.index >= index_below -5*60*25].min()
        time = int(df_decisions.iloc[i]['minute']) + int(df_decisions.iloc[i]['seconds'])/60
        #If there are only 7 minutes played in the first half, make sure you don't weight both 5 minutes the same
        if time - 45 < 10:
            mean_pc_b_home.append((5*sh.loc[index_below]['PC home previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['PC home previous 5 minutes'])/(time-45))
            mean_wpc_b_home.append((5*sh.loc[index_below]['Weighted PC home previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['Weighted PC home previous 5 minutes'])/(time-45))
            mean_pc_b_away.append((5*sh.loc[index_below]['PC away previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['PC away previous 5 minutes'])/(time-45))
            mean_wpc_b_away.append((5*sh.loc[index_below]['Weighted PC away previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['Weighted PC away previous 5 minutes'])/(time-45))
        else:
            mean_pc_b_home.append((5*sh.loc[index_below]['PC home previous 5 minutes'] + 5*sh.loc[index_5min_below]['PC home previous 5 minutes'])/10)
            mean_wpc_b_home.append((5*sh.loc[index_below]['Weighted PC home previous 5 minutes'] + 5*sh.loc[index_5min_below]['Weighted PC home previous 5 minutes'])/10)
            mean_pc_b_away.append((5*sh.loc[index_below]['PC away previous 5 minutes'] + 5*sh.loc[index_5min_below]['PC away previous 5 minutes'])/10)
            mean_wpc_b_away.append((5*sh.loc[index_below]['Weighted PC away previous 5 minutes'] + 5*sh.loc[index_5min_below]['Weighted PC away previous 5 minutes'])/10)            
        
        index_min_below.append(index_5min_below)
        index_min_above.append(index_5min_above)
        indexa.append(index_above)
        indexb.append(index_below)

    

    df_decisions['PC home 10min before'] = mean_pc_b_home
    df_decisions['WPC home 10min before'] = mean_wpc_b_home
    df_decisions['PC away 10min before'] = mean_pc_b_away
    df_decisions['WPC away 10min before'] = mean_wpc_b_away
    df_decisions['index 5min before'] = index_min_below
    df_decisions['index 5min after'] = index_min_above
    df_decisions['index below'] = indexb
    df_decisions['index above'] = indexa
    
    minutes_second_half = []
    
    #check if there are time windows where no decisions are made. If yes, add these time windows to the dataframe

    for i in range(len(df_decisions)):
        try: 
            if i == 0:
                if df_decisions.iloc[i]['index below'] - sh.iloc[0].name > 20*60*25:
                    dif = df_decisions.iloc[i]['index below'] - sh.iloc[0].name
                    if dif < 30*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/2)]['Time'])
                    elif dif >= 30*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/3)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/3)*2)]['Time'])                     

            if df_decisions.iloc[i+1]['index below'] - df_decisions.iloc[i]['index below'] > 20*60*25:
                dif = df_decisions.iloc[i+1]['index below'] - df_decisions.iloc[i]['index below']
                if dif < 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/2)]['Time'])
                elif dif >= 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int((dif/3)*2)]['Time'])

        except:
            if sh.iloc[-1].name - df_decisions.iloc[i]['index below'] > 20*60*25:
                dif = sh.iloc[-1].name - df_decisions.iloc[i]['index below']
                if dif < 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/2)]['Time'])
                elif dif >= 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index below'] + int((dif/3)*2)]['Time'])
    
                    
    mean_pc_b_home = []
    mean_wpc_b_home = []
    mean_pc_b_away = []
    mean_wpc_b_away = []
    index_min_above = []
    index_min_below = []
    indexa = []
    indexb = []
    minutes = []
    seconds = []
    half = []

    #for these new time windows, calculate the pitch control and PCV values
    for i in range(len(minutes_second_half)):
        minute = float(int(minutes_second_half[i]))
        second = float((minutes_second_half[i] - int(minutes_second_half[i]))*60)
        index_above = sh[sh['Time'] > float(minute + second/60)].iloc[0].name
        index_below = sh[sh['Time'] <= float(minute + second/60)].iloc[-1].name
        index_5min_below = sh.index[sh.index >= index_below -5*60*25].min()
        index_5min_above = sh.index[sh.index <= index_above +5*60*25].max()
        time = int(df_decisions.iloc[i]['minute']) + int(df_decisions.iloc[i]['seconds'])/60
        if time - 45 < 10:
            mean_pc_b_home.append((5*sh.loc[index_below]['PC home previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['PC home previous 5 minutes'])/(time-45))
            mean_wpc_b_home.append((5*sh.loc[index_below]['Weighted PC home previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['Weighted PC home previous 5 minutes'])/(time-45))
            mean_pc_b_away.append((5*sh.loc[index_below]['PC away previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['PC away previous 5 minutes'])/(time-45))
            mean_wpc_b_away.append((5*sh.loc[index_below]['Weighted PC away previous 5 minutes'] + (time-50)*sh.loc[index_5min_below]['Weighted PC away previous 5 minutes'])/(time-45))
        else:
            mean_pc_b_home.append((5*sh.loc[index_below]['PC home previous 5 minutes'] + 5*sh.loc[index_5min_below]['PC home previous 5 minutes'])/10)
            mean_wpc_b_home.append((5*sh.loc[index_below]['Weighted PC home previous 5 minutes'] + 5*sh.loc[index_5min_below]['Weighted PC home previous 5 minutes'])/10)
            mean_pc_b_away.append((5*sh.loc[index_below]['PC away previous 5 minutes'] + 5*sh.loc[index_5min_below]['PC away previous 5 minutes'])/10)
            mean_wpc_b_away.append((5*sh.loc[index_below]['Weighted PC away previous 5 minutes'] + 5*sh.loc[index_5min_below]['Weighted PC away previous 5 minutes'])/10)
        
        index_min_below.append(index_5min_below)
        index_min_above.append(index_5min_above)
        indexa.append(index_above)
        indexb.append(index_below)
        minutes.append(minute + 45)
        seconds.append(second)
        half.append(2)
    
    index = []
    type_id = []
    type_name = []
    half = []
    frame_id = []
    team_id = []
    team = []
    formation_change = []
    current_formation_home = []
    current_formation_away = []
    riskyness_decision = []
    match_id = []
    half = []
    minute = []
    seconds = []
    
    #add the other data for the new time windows
    for j in range(len(minutes_second_half)):
        index.append(df_decisions.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        match_id.append(df_decisions.iloc[0]['Match'])
        formation_change.append(0)
        frame_id.append(0)
        current_formation_home.append(0)
        current_formation_away.append(0)
        riskyness_decision.append('risk_neutral')
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)

    #create a dataframe for the new time windows
    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
            'formation_change' : formation_change,
            'current_formation_home' : current_formation_home,
            'current_formation_away' : current_formation_away,
            'Riskyness decision' : riskyness_decision,
           'Match': match_id,
            'PC home 10min before': mean_pc_b_home,
            'WPC home 10min before': mean_wpc_b_home, 
           'PC away 10min before': mean_pc_b_away,
            'WPC away 10min before': mean_wpc_b_away, 
            'index 5min before': index_min_below,
            'index 5min after': index_min_above,
           'index below': indexb,
           'index above': indexa}

    df = pd.DataFrame(data)
    #add the new time windows to the match dataframe
    df_new = pd.concat([df_decisions, df])
    
    #open the shots and goals dataframe
    #make a list with who scored in which minute
        
    infile = open('shots and goals event dataframes.p','rb')
    df_shot = pickle.load(infile)
    infile.close()
    goals = []
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['type_id'] == '16':
            goals.append([int(df_shots.iloc[i]['minute']) + int(df_shots.iloc[i]['seconds'])/60, df_shots.iloc[i]['team']])

    time_left = []
    home_goals = []
    away_goals = []
    #for every row in the dataframe: add the time left and the home and away score
    for i in range(len(df_new)):
        home = 0
        away = 0
        time_left.append(90 - (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60))
        for j in range(len(goals)):
            if (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Home':
                home = home + 1
            elif (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)

    df_new['Time left'] = time_left
    df_new['Home score'] = home_goals
    df_new['Away score'] = away_goals
    #add the new dataframe to the dataframe with all the matches
    final_df = pd.concat([final_df, df_new])

#create pickle file for all the matches that will be used as input for regression
pickle.dump(final_df, open('input PC regression Momentum on decisions.p', 'wb')) 


In [ ]:
#ten-minute xT calculations
from progressbar import ProgressBar
import pickle
import pandas as pd

#open decisions dataframe
infile = open('Decision matches dataframe.p','rb')
df_decisions = pickle.load(infile)
infile.close()

#open dataframe with xT for all matches per half minute
infile = open('all matches sliding windows xT.p','rb')
df_xT = pickle.load(infile)
infile.close()

#create an array with all matches in the xT dataframe
matches = df_xT.Match.unique()
pbar = ProgressBar()
final_df = pd.DataFrame()

#iterate over all matches in the xT dataframe
for game in pbar(matches):
    #only take the second half of each match in the dataframe
    match_decisions = df_decisions.loc[df_decisions['Match'] == game]
    match_decisions = match_decisions.loc[match_decisions['half'] == '2']
    
    #find time windows where no decision is made
    minutes_second_half = []
    for i in range(len(match_decisions)):
        try:
            if i == 0:
                if (int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60) - 45  > 20:
                    dif = (int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60) - 45
                    if dif < 30:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - dif/2)
                    elif dif >= 30:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - dif/3)
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - 2*dif/3)


            if (int(match_decisions.iloc[i+1]['minute'])+int(match_decisions.iloc[i+1]['seconds'])/60) - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60) > 20:
                dif = (int(match_decisions.iloc[i+1]['minute'])+int(match_decisions.iloc[i+1]['seconds'])/60) - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60)
                if dif < 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/3)

        except:
            if 91 - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60) > 20:
                dif = 91 - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60)
                if dif < 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/3)
        

    match_decisions = match_decisions[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'formation_change', 'current_formation_home', 'current_formation_away', 'Riskyness decision', 'Match']]

    index = []
    type_id = []
    type_name = []
    half = []
    frame_id = []
    team_id = []
    team = []
    formation_change = []
    current_formation_home = []
    current_formation_away = []
    riskyness_decision = []
    match_id = []
    half = []
    minute = []
    seconds = []

    #add other data for these new time windows
    for j in range(len(minutes_second_half)):
        index.append(match_decisions.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        match_id.append(match_decisions.iloc[0]['Match'])
        formation_change.append(0)
        frame_id.append(0)
        current_formation_home.append(0)
        current_formation_away.append(0)
        riskyness_decision.append('neutral')
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)

    #create a dataframe for the new time windows
    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
            'formation_change' : formation_change,
            'current_formation_home' : current_formation_home,
            'current_formation_away' : current_formation_away,
            'Riskyness decision' : riskyness_decision,
           'Match': match_id}

    df_new = pd.DataFrame(data)
    #add the dataframe of the new time windows to the match datafram
    df = pd.concat([match_decisions, df_new])
    
    #open the shots and goals dataframe
    infile = open('shots and goals event dataframes.p','rb')
    df_shot = pickle.load(infile)
    infile.close()
    goals = []
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    
    #create a list with each goal in the match and who scored
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['type_id'] == '16':
            goals.append([int(df_shots.iloc[i]['minute']) + int(df_shots.iloc[i]['seconds'])/60, df_shots.iloc[i]['team']])

    time_left = []
    home_goals = []
    away_goals = []
    
    #for each row in the match dataframe: add time left and home and away score
    for i in range(len(df)):
        home = 0
        away = 0
        time_left.append(90 - (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60))
        for j in range(len(goals)):
            if (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Home':
                home = home + 1
            elif (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)

    df['Time left'] = time_left
    df['Home score'] = home_goals
    df['Away score'] = away_goals
    
    #create the final dataframe but still without the xT calculations
    final_df = pd.concat([final_df, df])


In [ ]:
pbar = ProgressBar()
#add the xT values to the dataframe
df_final = pd.DataFrame()
#iterate over each match
for game in pbar(matches):
    #again, take only the second-half decisions
    df_decisions = final_df.loc[final_df['Match'] == game]
    df_decisions['minute'] = pd.to_numeric(df_decisions['minute'])
    df_decisions = df_decisions.loc[df_decisions['minute'] >= 50]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    df_xT_sh = df_match_xT.loc[df_match_xT['Half'] == 2]
    xt_home_10minb = []
    xt_away_10minb = []
    
    #add the xT values to the dataframe again making sure to weigh them properly 
    for i in range(len(df_decisions)):
        time = int(df_decisions.iloc[i]['minute']) + int(df_decisions.iloc[i]['seconds'])/60
        if time - 45 < 10:
            xt_home_10minb.append(5*(df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_5min_home']) + (time-50)*(df_xT_sh.loc[df_xT_sh['Minute'] <= time - 5].iloc[-1]['xT_prev_5min_home'])/(time-45))
            xt_away_10minb.append(5*(df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_5min_away']) + (time-50)*(df_xT_sh.loc[df_xT_sh['Minute'] <= time - 5].iloc[-1]['xT_prev_5min_away'])/(time-45))
        else:
            xt_home_10minb.append((df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_5min_home'] + df_xT_sh.loc[df_xT_sh['Minute'] <= time - 5].iloc[-1]['xT_prev_5min_home'])/2)
            xt_away_10minb.append((df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_5min_away'] + df_xT_sh.loc[df_xT_sh['Minute'] <= time - 5].iloc[-1]['xT_prev_5min_away'])/2)
            
    df_decisions['xT_prev_10min_home'] = xt_home_10minb
    df_decisions['xT_prev_10min_away'] = xt_away_10minb
    
    #for each match add the complete dataframe to the final dataframe
    df_final = pd.concat([df_decisions, df_final])

#export the complete dataframe for all matches to a pickle file to be used for regression
pickle.dump(df_final, open('input xT regression Momentum on decisions.p', 'wb')) 